## Importing Library

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras import backend as K

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

print(tf.__version__)

Using TensorFlow backend.


1.10.0


## Import Sample Data

In [2]:
path = "/Users/chanhee/Desktop/foundation_toy.csv"
idx_col = 0

data = pd.read_csv(path,index_col=idx_col)
sample,features = data.shape


y_val = data.result
x_val = data.drop(["patient","cancer_code","result"],axis=1)
print("sample : {}  \nfeatures : {}".format(sample,features))

sample : 1400  
features : 311


## train each model for ensemble

In [3]:
from keras.layers import Input, Dense
from keras.models import Model

input_m1 = Input(shape=(308,))
h1_m1 = Dense(200,activation='relu')(input_m1)
h2_m1 = Dense(150,activation='relu')(h1_m1) 
h3_m1 = Dense(100,activation='relu')(h2_m1) 
h4_m1 = Dense(10,activation='relu')(h3_m1) 
output_m1 = Dense(1,activation="sigmoid")(h4_m1) 
model1 = Model(inputs=input_m1,outputs=output_m1)


input_m2 = Input(shape=(308,))
h1_m2 = Dense(250,activation='relu')(input_m2)
h2_m2 = Dense(100,activation='relu')(h1_m2) 
h3_m2 = Dense(30,activation='relu')(h2_m2)
h4_m2 = Dense(10,activation='relu')(h3_m2) 
output_m2 = Dense(1,activation="sigmoid")(h4_m2)
model2 = Model(inputs=input_m2,outputs=output_m2)

input_m3 = Input(shape=(308,))
h1_m3 = Dense(200,activation='relu')(input_m3)
h2_m3 = Dense(100,activation='relu')(h1_m3) 
h3_m3 = Dense(100,activation='relu')(h2_m3)
h4_m3 = Dense(10,activation='relu')(h3_m3) 
output_m3 = Dense(1,activation="sigmoid")(h4_m3)
model3 = Model(inputs=input_m3,outputs=output_m3)

input_m4 = Input(shape=(308,))
h1_m4 = Dense(200,activation='relu')(input_m4)
h2_m4 = Dense(100,activation='relu')(h1_m4) 
h3_m4 = Dense(100,activation='relu')(h2_m4)
h4_m4 = Dense(10,activation='relu')(h3_m4) 
output_m4 = Dense(1,activation="sigmoid")(h4_m4)
model4 = Model(inputs=input_m4,outputs=output_m4)

input_m5 = Input(shape=(308,))
h1_m5 = Dense(200,activation='relu')(input_m5)
h2_m5 = Dense(10,activation='relu')(h1_m5) 
h3_m5 = Dense(150,activation='relu')(h2_m5)
h4_m5 = Dense(10,activation='relu')(h3_m5) 
output_m5 = Dense(1,activation="sigmoid")(h4_m5)
model5 = Model(inputs=input_m5,outputs=output_m5)

input_m6 = Input(shape=(308,))
h1_m6 = Dense(200,activation='relu')(input_m6)
h2_m6 = Dense(150,activation='relu')(h1_m6) 
h3_m6 = Dense(10,activation='relu')(h2_m6)
h4_m6 = Dense(10,activation='relu')(h3_m6) 
output_m6 = Dense(1,activation="sigmoid")(h4_m6)
model6 = Model(inputs=input_m6,outputs=output_m6)

model1.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])
model2.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])
model3.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])
model4.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])
model5.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])
model6.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model1.fit(x_val, y_val, epochs=10)
model2.fit(x_val, y_val, epochs=10)
model3.fit(x_val, y_val, epochs=10)
model4.fit(x_val, y_val, epochs=10)
model5.fit(x_val, y_val, epochs=10)
model6.fit(x_val, y_val, epochs=10)

Epoch 1/10
1400/1400 [==============================] - 1s 446us/step - loss: 0.3950 - acc: 0.8164
Epoch 2/10
1400/1400 [==============================] - 0s 64us/step - loss: 0.2235 - acc: 0.9029
Epoch 3/10
1400/1400 [==============================] - 0s 66us/step - loss: 0.1738 - acc: 0.9307
Epoch 4/10
1400/1400 [==============================] - 0s 65us/step - loss: 0.1409 - acc: 0.9464
Epoch 5/10
1400/1400 [==============================] - 0s 66us/step - loss: 0.1259 - acc: 0.9571
Epoch 6/10
1400/1400 [==============================] - 0s 66us/step - loss: 0.1251 - acc: 0.9464
Epoch 7/10
1400/1400 [==============================] - 0s 64us/step - loss: 0.0850 - acc: 0.9686
Epoch 8/10
1400/1400 [==============================] - 0s 67us/step - loss: 0.0726 - acc: 0.9707
Epoch 9/10
1400/1400 [==============================] - 0s 65us/step - loss: 0.0577 - acc: 0.9793
Epoch 10/10
1400/1400 [==============================] - 0s 67us/step - loss: 0.0443 - acc: 0.9850
Epoch 1/10
1400/14

## Ensembel Coverage Algorithm

In [4]:
def ensemble_coverage(inputModels,x,y):
    outputModels = []
    modelInfo = []
    coverageTotal= [False]*len(y)
    
    for m in inputModels:
        yHat = m.predict(x)
        yHat = [round(i) for [i] in yHat]
        
        loss, acc = m.evaluate(x,y)
        modelInfo.append((m,yHat,acc))
    
    modelInfo.sort(key=lambda x : x[2],reverse=True)
    
    for m,yHat,acc in modelInfo:
        beforeCoverage = sum(coverageTotal)
        coverage = [a == b for a,b in zip(y,yHat)]
        coverageTotal = [a or b for a,b in zip(coverageTotal,coverage)]
        afterCoverage = sum(coverageTotal)
        
        print(afterCoverage/len(y))
        
        if afterCoverage > beforeCoverage:
            outputModels.append(m)
            print("Increased Coverage : model added!")
        else:
            print("Same Coverage : model not added")
        if afterCoverage == len(y):
            print("Fully Covered!")
            break
        
                
    return outputModels


## Getting Maximum Coverage Model Combinations

In [7]:
inputModels = [model1,model2,model3,model4,model5,model6]
x = x_val
y = y_val

ensemble_models = ensemble_coverage(inputModels,x,y)
print("number of final models for ensemble: ",len(ensemble_models))

1400/1400 [==============================] - 0s 26us/step
0.9942857142857143
Increased Coverage : model added!
0.9957142857142857
Increased Coverage : model added!
0.9957142857142857
Same Coverage : model not added
0.9978571428571429
Increased Coverage : model added!
0.9985714285714286
Increased Coverage : model added!
0.9985714285714286
Same Coverage : model not added
number of final models for ensemble:  4
